In [37]:
# inspired by
# https://github.com/fjg00/Facebook-Group-Post-Scraper/blob/main/Facebook%20Group%20Parser.py
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

import argparse
from datetime import datetime
import time
import json

import os
import re
import pandas as pd

In [38]:
# USER DEFINED INPUTS
import Login
CHROME_DRIVER_PATH = './chromedriver' # must download this yourself

# EMAIL, PASSWORD from Login file (explained in README.md)
EMAIL = Login.EMAIL
PASSWORD = Login.PASSWORD

# Group number
GROUP_ID = 1379345962387168 #USER INPUT
GROUP_URL = 'https://www.facebook.com/groups/'+str(GROUP_ID)

SEARCH_PROMPT = "" # something to be searched for in the group
SEARCH_PROMPT = SEARCH_PROMPT.replace(" ","%20")

# Copy the group description and put it here to be able to filter it out
GROUP_DESCRIPTION = """-Kamer in Eindhoven-
Kamer in Eindhoven is een platform waar vraag en aanbod (kosteloos) worden samengebracht. Ben je opzoek naar een kamer, huisgenoot of bied je een kamer aan, word dan nu lid van deze facebookgroep.
Je scrollt door de nieuwste kamers en huisgenoten om vervolgens direct te reageren naar de aanbieder. Bij ons geen inschrijfkosten, gewoon studenten onder elkaar.
Kamer in Eindhoven is onderdeel van de Facebook community Zoekt Kamer in… Wij zijn ook actief in Nijmegen, Amsterdam, Delft, Rotterdam, Groningen, Maastricht, Breda, Utrecht, Leiden, Den Haag, Haarlem en Amersfoort.
Sinds 2020 hebben wij een samenwerking met de gratis app MyHospi. Hiermee willen wij het proces voor jullie nog makkelijker maken.
HOE WERKT MYHOSPI VOOR HUIZEN?
1. Plaats de kamer op myHospi via de app.
2. Deel de link naar het huis in je FB bericht en geef aan dat mensen via myHospi (de link) moeten reageren. Met myHospi krijg je direct een compleet beeld van de mensen die reageren op jouw kamer. Geen onpersoonlijke mails maar complete profielen.
3. Swipe en Like!
4. Nodig kandidaten uit en deel de link naar je videocall.""" # User input

In [39]:
def addvalues(dictionary: dict, key, L: list):
    """Append multiple values to a key in the given dictionary"""
    if key not in dictionary:
        dictionary[key] = list()
    dictionary[key].extend(L)
    return dictionary


In [40]:
# Webdriver options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('headless')

driver = webdriver.Chrome(CHROME_DRIVER_PATH, options=chrome_options) #USER INPUT
driver.get('https://www.facebook.com/')

WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='_42ft _4jy0 _9xo6 _4jy3 _4jy1 selected _51sy']"))).click()
email = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name = 'email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name = 'pass']")))


email.clear()
password.clear()

email.send_keys(EMAIL) #USER INPUT
password.send_keys(PASSWORD) #USER INPUT

/tmp/ipykernel_41316/3073893559.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(CHROME_DRIVER_PATH, options=chrome_options) #USER INPUT


In [41]:
# Open window, select group
login = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[type = 'submit']"))).click()

#maximize window
driver.maximize_window()

n_scrolls = 2

# Gets FB group with search parameters
driver.get(GROUP_URL+SEARCH_PROMPT)

In [42]:
# Selection criteria
text = ""
comments = ""
z = 1

In [43]:
# Scroll through the page and get all links
def extract(comment):
    return comment.partition('?comment')[0]

def getPostURLs(limit: int):
    """ Get a list of post URLs from the Facebook group of choice.

    Args: 
        limit (int): number of post URLs that should be returned.

    Returns:
        posts (list): list of post URLs.
    """

    all_urls = []
    posts_number = 0
    posts = []

    while len(posts) <= limit:
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        time.sleep(5)
        
        a_tags = driver.find_elements(By.TAG_NAME, 'a')
        new_urls = [a.get_attribute('href') for a in a_tags]

        # New URLs should be of post or comment type (which is a subtype of a post URL). 
        new_comments = [a for a in new_urls if (str(a).startswith(GROUP_URL+'/posts/'))]

        # Only extract the post URL, not interested in the comment links.
        new_posts = set(extract(comment) for comment in new_comments if extract(comment) not in posts)
        posts.extend(list(new_posts))

        # Print progress
        if len(posts) < limit:
            print ('collected', len(posts))
        else:
            print ('collected:', limit)
    
    return posts[:limit]

In [44]:
posts = getPostURLs(5)

collected 2
collected 4
collected: 5


In [45]:
posts[3]

'https://www.facebook.com/groups/1379345962387168/posts/3371879229800488/'

In [94]:
def getPosterURL(post: str, page_source=None, soup=None):
    driver.get(post)
    time.sleep(1)
    poster_class = "x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz xt0b8zv xzsf02u x1s688f"

    link = driver.page_source if page_source is None else page_source
    page_source = 'https://www.facebook.com' + page_source if link is None else link


    soup = BeautifulSoup(page_source, 'html.parser') if soup is None else soup

    urls = soup.find_all('a', class_=poster_class)
    urls = [url['href'] for url in urls]

    identifier = "/groups/" + str(GROUP_ID)
    url = [url for url in urls if identifier in url][0]
    url = "https://www.facebook.com" + str(url)

    return url

In [98]:
def getPosterList(post_list: list):
    urls = []
    for post in post_list:
        urls.append(getPosterURL(post))
    return urls

In [99]:
posters = getPosterList(posts)

In [100]:
posters

['https://www.facebook.com/groups/1379345962387168/user/100002814206181/?__cft__[0]=AZVLXhkM5P9QGTQyt5SkPrL_oRiGnePcxARwEEqaP36Kw_tPbM7GEoTYrs5_tdDqofCdo49KH_RuYWUG9mrlTQOjFpJAjjDDOZ9D3ojmhXOiftFDRyOHeryEz8FhW7vm3TnCF7DplkNX1wBtXrMMjPJE&__tn__=-UC%2CP-R',
 'https://www.facebook.com/groups/1379345962387168/user/100019688293351/?__cft__[0]=AZXUOIs8aZx2fGjMJY78YuvK2sfZudxxwN4mJRwbEgvbUPzJPm-UytX5voB0YL-s2asUlWsfyMSP1I6t8UZNrOrvc5EThHoi4_1NDD5hNHZwyympyuNpK-7KIpMj18neaSXXwUTyVC6fMDznljYJO6GW&__tn__=-UC%2CP-R',
 'https://www.facebook.com/groups/1379345962387168/user/100009870044900/?__cft__[0]=AZWXcbgVUmXh_OndQ1q1wwYl7cZPnriZUcapH4qQTOHJaBsKzuezO-ZyQ7Vj8dCtiEU17L6czvTrSO1Gvpv-XFE6WG1yTbuKSGZIHnZfIk97oiqINmSHKvK0B7y0zH9001NI8_szdOBSJkNYQF921_nv&__tn__=-UC%2CP-R',
 'https://www.facebook.com/groups/1379345962387168/user/100074367884538/?__cft__[0]=AZXvZoCfGwNGOZLgJUZjQvY-1-EsYEoxqXimhofjYI-KZWv2U7iIs52PDpGMqoCrq2FGnCW9ONwr5RDzVH-E_LtyI0tOo4BeDtq0cfvh17XPTvADyNvqEWZw7StBdbO72I6QKUiGYt1IbJZcvLW3

In [47]:
# # Grab post and comment data
# # TODO make nice df out of it
# # TODO grab info about posters and commenters
# # TODO grab time data (and time difference between post and comment)


# WrapperDict = list() 
# texts = []
# translatedBools = []
# commentCounts = []

# for postCount, a in enumerate(posts):
#         driver.get(a)
#         time.sleep(1)
        
#         page_source = driver.page_source
#         soup = BeautifulSoup(page_source, 'html.parser')
        
#         # Relevant classes for text (or translate button)
#         translateBtnText = "x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x1n2onr6 x87ps6o x1a2a7pz xt0b8zv"
#         translateBtn = soup.find('div', class_ = translateBtnText)
#         translatedTextClass = "x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x6prxxf xvq8zen xo1l8bm xzsf02u x1yc453h"
#         originalTextClass = "x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x41vudc x6prxxf xvq8zen xo1l8bm xzsf02u x1yc453h"

#         if (translateBtn is not None) and (soup.find('div', class_ = translateBtnText).text == "See Translation"):
#                 isTranslated = False
#                 # Original
#                 # text = soup.find('span' , class_ = originalTextClass).text #post
#                 t = soup.find('div', class_ = "x1swvt13 x1l90r2v x1pi30zi x1iorvi4")
#                 if t is not None:
#                         text = t.text #post
#                 else:
#                         # This may result in text="Facebook", could find all of this class and choose one where it doesn't equal Facebook
#                         text = soup.find('span' , class_ = originalTextClass).text #post
#         elif soup.find('span', class_ = translatedTextClass) is not None:
#                 # Translated
#                 text = soup.find('span' , class_ = translatedTextClass).text #post
#                 isTranslated = True
#         elif soup.find('span', class_ = originalTextClass) is not None:
#                 # Original English
#                 text = soup.find('span', class_ = originalTextClass).text #post
#                 isTranslated = False

#         # isTranslated = len(soup.find_all('span', class_ = "x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x1pg5gke xvq8zen xo1l8bm x1qq9wsj x1yc453h")) > 0
#         # TODO instead of length check that it equals expected text ('See original' or 'Rate this translation')

#         commentCount = soup.find('span', class_ = "x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x41vudc x6prxxf xvq8zen xo1l8bm xi81zsa").text
#         commentCount = int(re.search(r'\d+', commentCount).group())
        
#         # TODO create DF/dictionary/smth with all this info
#         texts.append(text)
#         translatedBools.append(isTranslated)
#         commentCounts.append(commentCount)
#         if postCount % 5 == 0:
#                 print (postCount)

#         # w = 1
        
#         # BranchDict = dict()
#         # BranchDict["tag"] = 1
#         # BranchDict["patterns"] = 1
#         # BranchDict["responses"] = list()
#         # L = list()
        
#         # for post in actualPosts:
#         #     s = post.get_text()
#         #     # if (s == "INSERT GROUP DESCRIPTION HERE"): #USER INPUT
#         #     if (s == GROUP_DESCRIPTION): #USER INPUT
#         #         time.sleep(1)
#         #     elif len(s.split()) < 4:
#         #         time.sleep(1)
#         #     else:
#         #         if w == 1: #post
#         #             text= text + post.get_text() 
#         #             BranchDict["tag"] = SEARCH_PROMPT + str(z) 
#         #             BranchDict["patterns"] = text
#         #             w = w + text1
#         #         elif w > 1 : #comments
#         #             comments = post.get_text()
#         #             L.append(comments)
#         #             comments = ""
#         # z = z + 1
#         # addvalues(BranchDict, "responses", L)
#         # WrapperDict.append(BranchDict)
#         # text = ""

In [48]:
# Close the headless session (to avoid it staying alive in memory)
# driver.quit()

In [49]:
# # df = pd.DataFrame([texts, translatedBools, commentCounts], columns=['Text', 'Is_Translated', 'Comment_Count'])
# df = pd.DataFrame.from_dict({"Text": texts, "Is_Translated": translatedBools, "Comment_Count": commentCounts, 'URL': list(posts)})

# # # Flatten text if necessary
# # def flatten(text):
# #     if text is list:
# #         text = ''.join(text)
# #         print (text)
# #     return text
# # df['Text'] = df['Text'].apply(flatten)
# # df.iloc[0]['URL']
# df

In [50]:
# df.describe()

In [51]:
# Save it, brother
# df.to_csv(path_or_buf='scraped_data.csv', index=False)